##  Comparing the titles of publication in a curated listgenerated by the Bodelian Libraries team with titles gathered from searches of CrossRef API and data from Dimensions.ai related the BRC Oxford phase 2

### The original file had been curated and checked.  Reasons why differences between this and the Dimensions-aquired list may differ:
- no mention of NIHR anywhere in paper?
- dimensions list includes mentions in the acknowledgements?
- match not found with DOI or title due to text errors in list
- publication type is not and article?

### Furthermore, the original titles ('title_post_csv') was used to search for the most likely match (top hit) from CrossRef..  if acquiring missing DOIs in this way works then the ' title' retrived using the CrossRef API a the same time should match...

#### There are, however many reasons why they might not match exactly:


       

In [1]:
'hypoxia regulates FGFR3 expression via HIF-1alpha...' == 'Hypoxia regulates FGFR3 expression via HIF-1alpha...'

False

In [2]:
'Hypoxia regulates FGFR3 expression via HIF-1alpha...' == 'Hypoxia regulates FGFR3 expression via HIF-1alpha ...'

False

In [3]:
'hypoxia regulates FGFR3 expression via HIF-1alpha...' == 'Hypoxia regulates FGFR3 expression via HIF-1α...'

False

## In this notebook we will look at matching the two lists by title and DOI, both for exact matches and with a Fuzzy match (using the fuzz function from the Fuzzwuzzy python library)

Does fuzzy matching produce more matched records than just converting strings to lower case and stripping any additional white space??

In [103]:
import pandas as pd  # dataframes - importing csv files
import numpy as np
from fuzzywuzzy import fuzz, process



In [104]:
# Import our original file with CrossREF data added (book A2):
df1 = pd.read_csv('./A2out_CrossRef_BRC_matched_and_unmmatched_06Aug19.csv', index_col=0)
df1.head()

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match
0,10.1186/s12881-014-0095-4,1125,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",{'title': ['Apparently synonymous substitution...,Apparently synonymous substitutions in FGFR2 a...,10.1186/s12881-014-0095-4,['Apparently synonymous substitutions in FGFR2...,Apparently synonymous substitutions in FGFR2af...,False,True
1,10.1183/13993003.00321-2016,1996,", Pattinson KT, Turner MR. A wider pathologic...",{'title': ['A wider pathological network under...,A wider pathological network underlying breath...,10.1183/13993003.00321-2016,['A wider pathological network underlying brea...,A wider pathological network underlying breath...,True,True
2,NaN,506,"Adib-Samii P, Rost N, Traylor M, Devan W, ...",{'title': ['17q25 Locus Is Associated With Whi...,17q25 Locus is associated with white matter hy...,10.1161/strokeaha.113.679936,['17q25 Locus Is Associated With White Matter ...,17q25 Locus Is Associated With White Matter Hy...,True,False
3,10.1093/annonc/mdu449,1430,"and I. Tomlinson*, Findlay JM, Middleton MR, ...",{'title': ['A systematic review and meta-analy...,A systematic review and meta-analysis of somat...,10.1093/annonc/mdu449,['A systematic review and meta-analysis of som...,A systematic review and meta-analysis of somat...,True,True
4,NaN,848,"Dichgans M, Malik R, KÃ•_nig IR, Rosand J,...",{'title': ['Shared Genetic Susceptibility to I...,Shared genetic susceptibility to ischemic stro...,10.1161/strokeaha.113.002707,['Shared Genetic Susceptibility to Ischemic St...,Shared Genetic Susceptibility to Ischemic Stro...,False,False


In [105]:
#remove known differences due to whitespace and differences due to UPPER/Title/lower cases
df1['complete']= df1['complete'].str.strip().str.lower()
df1['csv_post_title']= df1['csv_post_title'].str.strip().str.lower()
df1['CrossRefTitle_low']= df1['title_API'].str.strip().str.lower()

In [106]:
df1.loc[331]

DOI                                       10.1371/journal.pone.0102645
ID                                                                1108
complete             chambers jc, abbott j, zhang w, turro e, scott...
crossref_API_out     {'title': ['Utility Sector Performance Post Re...
csv_post_title                                  the south asian genome
DOI_API                                                10.1002/pop4.68
title                ['Utility Sector Performance Post Reforms: Inv...
title_API            Utility Sector Performance Post Reforms: Inves...
title_match                                                      False
DOI_match                                                        False
CrossRefTitle_low    utility sector performance post reforms: inves...
Name: 331, dtype: object

In [107]:
def doiFuzzRatio(df):
    if (pd.notna(df.DOI))&(pd.notna(df.DOI_API)):
        ratio = fuzz.ratio(df.DOI, df.DOI_API)
        return ratio
    else:
        return None

In [121]:
def titleFuzzRatio(df):
    if (pd.notnull(df.csv_post_title)& pd.notnull(df.title_API)):
        ratio = fuzz.ratio(df.csv_post_title, df.CrossRefTitle_low)
        part = fuzz.partial_ratio(df.csv_post_title, df.CrossRefTitle_low)
        return ratio, part
    else:
        return None

In [122]:
def completeFuzzRatio(df):
    if (pd.notnull(df.complete))&(pd.notnull(df.title_API)):
        ratio = fuzz.ratio(df.complete, df.CrossRefTitle_low)
        part = fuzz.partial_ratio(df.complete, df.CrossRefTitle_low)
        return ratio, part
    else:
        return None

In [123]:
df1[df1.CrossRefTitle_low.str.len()<=10]

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01


In [124]:
df1['DOI_fuzzyRatio'] = df1.apply(doiFuzzRatio, axis=1)
df1[['Title_fuzzyRatio','Title_partialRatio']]= df1.apply(titleFuzzRatio, axis=1, result_type='expand')
df1[['Complete_fuzzyRatio','Complete_partialRatio']]= df1.apply(completeFuzzRatio, axis=1, result_type='expand')

In [125]:
df1.dtypes

DOI                       object
ID                         int64
complete                  object
crossref_API_out          object
csv_post_title            object
DOI_API                   object
title                     object
title_API                 object
title_match                 bool
DOI_match                   bool
CrossRefTitle_low         object
DOI_fuzzyRatio           float64
TitleMatch01               int32
Title_fuzzyRatio         float64
Title_partialRatio       float64
Complete_fuzzyRatio      float64
Complete_partialRatio    float64
dtype: object

In [126]:
import hvplot as hvp
import hvplot.pandas
import holoviews as hv


In [127]:
df1.DOI_fuzzyRatio.hvplot(kind='scatter')

:Scatter   [index]   (DOI_fuzzyRatio)

In [128]:
df1.Title_fuzzyRatio.hvplot.scatter()

:Scatter   [index]   (Title_fuzzyRatio)

In [132]:
df1['TitleMatch01']= df1.title_match.astype(int)

df1.hvplot.scatter(x='Title_fuzzyRatio', y='Complete_partialRatio', height=400, width=600,
                    c='DOI_fuzzyRatio',cmap='Rainbow',hover_cols=['ID','csv_post_title','CrossRefTitle_low'],
                  xlim=(0,105),ylim=(0,105), alpha=0.4)
# adding columns to the hover tool allows judgement over where correct matchin has taken place.

:Scatter   [Title_fuzzyRatio]   (Complete_partialRatio,DOI_fuzzyRatio,ID,csv_post_title,CrossRefTitle_low)

In [130]:
#good_list = df1[((df1.Title_fuzzyRatio >= 80)| (df1.Title_partialRatio >= 95)|(df1.Complete_partialRatio >= 95)| (df1.DOI_match==True))]
#check_list = df1[~((df1.Title_fuzzyRatio >= 80)| (df1.Title_partialRatio >= 95) | (df1.Complete_partialRatio >= 95)| (df1.DOI_match==True))]

In [135]:
df1['good_list'] = ((df1.Title_fuzzyRatio >= 80)| (df1.Title_partialRatio >= 95)|(df1.Complete_partialRatio >= 95)| (df1.DOI_match==True))

In [136]:
df1.groupby('good_list').count()

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio
good_list,,,,,,,,,,,,,,,,,
False,221,379,379,379,379,379,379,353,379,379,353,221,379,353,353,353,353
True,1219,1999,1999,1999,1999,1999,1999,1999,1999,1999,1999,1219,1999,1999,1999,1999,1999


In [137]:
#check_list to file
df1[df1.good_list==False].to_csv('./A3out_to_check_bod_list_vs_CrossRef_mismatches(07Aug19).csv')

####  ~~first check showed CrossRef API-derived titles occasionally have  "double" quotes rather then 'single'  redo regex to correct~~
#### Corrected

In [138]:
df1.to_csv('./A3out_CrossRef_BRC_FuzzyMatched_07Aug19.csv')

In [139]:
df1.head()

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list
0,10.1186/s12881-014-0095-4,1125,"&amp; , fenwick al, goos jac, rankin j, lord h...",{'title': ['Apparently synonymous substitution...,apparently synonymous substitutions in fgfr2 a...,10.1186/s12881-014-0095-4,['Apparently synonymous substitutions in FGFR2...,Apparently synonymous substitutions in FGFR2af...,False,True,apparently synonymous substitutions in fgfr2af...,100.0,0,99.0,99.0,51.0,99.0,True
1,10.1183/13993003.00321-2016,1996,", pattinson kt, turner mr. a wider pathologica...",{'title': ['A wider pathological network under...,a wider pathological network underlying breath...,10.1183/13993003.00321-2016,['A wider pathological network underlying brea...,A wider pathological network underlying breath...,True,True,a wider pathological network underlying breath...,100.0,1,100.0,100.0,73.0,100.0,True
2,NaN,506,"adib-samii p, rost n, traylor m, devan w, ...",{'title': ['17q25 Locus Is Associated With Whi...,17q25 locus is associated with white matter hy...,10.1161/strokeaha.113.679936,['17q25 Locus Is Associated With White Matter ...,17q25 Locus Is Associated With White Matter Hy...,True,False,17q25 locus is associated with white matter hy...,NaN,1,100.0,100.0,25.0,100.0,True
3,10.1093/annonc/mdu449,1430,"and i. tomlinson*, findlay jm, middleton mr, t...",{'title': ['A systematic review and meta-analy...,a systematic review and meta-analysis of somat...,10.1093/annonc/mdu449,['A systematic review and meta-analysis of som...,A systematic review and meta-analysis of somat...,True,True,a systematic review and meta-analysis of somat...,100.0,1,100.0,100.0,72.0,100.0,True
4,NaN,848,"dichgans m, malik r, kã•_nig ir, rosand j, ...",{'title': ['Shared Genetic Susceptibility to I...,shared genetic susceptibility to ischemic stro...,10.1161/strokeaha.113.002707,['Shared Genetic Susceptibility to Ischemic St...,Shared Genetic Susceptibility to Ischemic Stro...,False,False,shared genetic susceptibility to ischemic stro...,NaN,0,78.0,100.0,14.0,100.0,True


In [140]:
df_compete_check =df1[df1.good_list==False]

In [141]:
#df_compete_check = pd.read_clipboard(sep='\n')

In [142]:
df_compete_check

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list
5,NaN,1312,"g w, rane , harston g w, rane n, shaya g, t...",{'title': ['Imaging-based selection in acute i...,imaging biomarkers in acute ischemic stroke tr...,10.1111/j.1749-6632.2012.06732.x,['Imaging-based selection in acute ischemic st...,Imaging-based selection in acute ischemic stro...,False,False,imaging-based selection in acute ischemic stro...,NaN,0,68.0,72.0,43.0,64.0,False
7,10.1016/j.atherosclerosis.2012.03.036,62,"kylintireas i, craig s, nethononda r, kohle...",{'title': ['Obstructive sleep apnea in childre...,atherosclerosis and arterial stiffness in obst...,10.1002/lary.23844,['Obstructive sleep apnea in children'],Obstructive sleep apnea in children,False,False,obstructive sleep apnea in children,40.0,0,42.0,77.0,23.0,77.0,False
10,NaN,168,"pagnamenta at, murray je, yoon g, akha es, ...",{'title': ['High-frequency sensorineural heari...,a novel nonsense cdk5rap2 mutation in a somali...,10.1002/lary.25544,['High-frequency sensorineural hearing loss in...,High-frequency sensorineural hearing loss in c...,False,False,high-frequency sensorineural hearing loss in c...,NaN,0,41.0,60.0,24.0,62.0,False
20,10.1016/j.fertnstert.2014.12.123,1920,"åukaszuk k, pukszta s, wells d, cybulska c, l...",{'title': ['Validation of a semiconductor next...,routine use of next-generation sequencing for ...,10.1002/pd.4665,['Validation of a semiconductor next-generatio...,Validation of a semiconductor next-generation ...,False,False,validation of a semiconductor next-generation ...,34.0,0,62.0,64.0,42.0,64.0,False
21,NaN,1023,"abdel-hady m, bansal s, davison sm, brown m, t...",{'title': ['Chronic hepatitis C in children - ...,treatment of chronic viral hepatitis c in chil...,10.1111/j.1365-2893.2011.01456.x,['Chronic hepatitis C in children - review of ...,Chronic hepatitis C in children - review of na...,False,False,chronic hepatitis c in children - review of na...,NaN,0,55.0,59.0,35.0,56.0,False
26,10.3389/fmicb.2015.01016,1642,"adland e, klenerman p, goulder p, matthews pc....",{'title': ['Review of cytomegalovirus coinfect...,ongoing burden of disease and mortality from h...,10.1002/rmv.1907,['Review of cytomegalovirus coinfection in HIV...,Review of cytomegalovirus coinfection in HIV-i...,False,False,review of cytomegalovirus coinfection in hiv-i...,40.0,0,47.0,52.0,32.0,52.0,False
33,NaN,1639,"ahmed s, brennan l, eppig j, price cc, lamar m...",{'title': ['Comparative study between depressi...,visuoconstructional impairment in subtypes of ...,10.1111/j.1442-2018.2011.00666.x,['Comparative study between depression in Kore...,Comparative study between depression in Korean...,False,False,comparative study between depression in korean...,NaN,0,47.0,56.0,38.0,49.0,False
55,10.1097/BOR.0b013e3283620163,563,"al-mossawi mh, ridley a, kiedel s, bowness p. ...",{'title': ['Gamma delta T cells in non-immune ...,"the role of natural killer cells, gamma delta ...",10.1002/iid3.18,['Gamma delta T cells in non-immune patients d...,Gamma delta T cells in non-immune patients dur...,False,False,gamma delta t cells in non-immune patients dur...,42.0,0,47.0,54.0,33.0,55.0,False
72,10.3233/jpd-120126,490,"antoniades c, xuc z, carpenter rhs, barker r. ...",{'title': ['The Relationship Between Brain Agi...,the relationship between abnormalities of sacc...,10.3724/sp.j.1206.2012.00351,['The Relationship Between Brain Aging and Pre...,The Relationship Between Brain Aging and Precl...,False,False,the relationship between brain aging and precl...,52.0,0,64.0,62.0,46.0,62.0,False
74,NaN,66,"antoniades ca, carpenter rhs. making neurology...",{'title': ['Initiating decision-making in neur...,making neurology quantitative,10.1111/1467-9566.12000,['Initiating decision-making in neurology cons...,Initiating decision-making in neurology c

  https://apps.crossref.org/simpleTextQuery